In [ ]:
import os
import kagglehub
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import copy

In [2]:
# Download latest version
path = kagglehub.dataset_download("Cornell-University/arxiv")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Salman Awaise\.cache\kagglehub\datasets\Cornell-University\arxiv\versions\226


In [3]:
files = os.listdir(path)
print(files)

['arxiv-metadata-oai-snapshot.json']


In [ ]:
# Utility: Clone layers
def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [ ]:
# Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Creating constant 'pe' matrix
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # [1, max_len, d_model]
        self.register_buffer('pe', pe)

    def forward(self, x):
        # Adding positional encoding
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

In [ ]:
# Layer Normalization
class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [ ]:
# Sublayer Connection (Residual + Norm)
class SublayerConnection(nn.Module):
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))

In [ ]:
# Multi-Head Attention
def attention(query, key, value, mask=None, dropout=None):
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = torch.softmax(scores, dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [ ]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        if mask is not None:
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        query, key, value = [
            l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
            for l, x in zip(self.linears, (query, key, value))
        ]

        x, self.attn = attention(query, key, value, mask, self.dropout)
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

In [ ]:
# Position-wise Feed Forward
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [ ]:
# Encoder Layer
class EncoderLayer(nn.Module):
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

In [ ]:
# Decoder Layer
class DecoderLayer(nn.Module):
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, src_mask, tgt_mask):
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [ ]:
# Encoder and Decoder Stacks
class Encoder(nn.Module):
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class Decoder(nn.Module):
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [ ]:
# Embeddings with Positional Encoding
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [ ]:
# Full model
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)

    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [ ]:
# Generator
class Generator(nn.Module):
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

# Mask to block future positions
def subsequent_mask(size):
    attn_shape = (1, size, size)
    mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(torch.uint8)
    return mask == 0

In [ ]:
# Build full model
def make_model(vocab_size, N=4, d_model=512, d_ff=2048, h=8, dropout=0.1):
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, attn, ff, dropout), N),
        Decoder(DecoderLayer(d_model, attn, attn, ff, dropout), N),
        nn.Sequential(Embeddings(d_model, vocab_size), position),
        nn.Sequential(Embeddings(d_model, vocab_size), position),
        Generator(d_model, vocab_size)
    )
    return model


In [5]:
from transformers import T5Tokenizer
import pandas as pd
import torch

# Loading T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")

# Loading the dataset (use your downloaded path)
df = pd.read_json(path+'/arxiv-metadata-oai-snapshot.json', lines=True)
df = df[df['abstract'].notna() & df['title'].notna()]
df = df.head(1000)  # Limit for testing

# Tokenizing a row
def tokenize_row(row):
    input_text = "summarize: " + row["abstract"]
    target_text = row["title"]

    input_enc = tokenizer(
        input_text, max_length=512, padding="max_length", truncation=True, return_tensors="pt"
    )
    target_enc = tokenizer(
        target_text, max_length=64, padding="max_length", truncation=True, return_tensors="pt"
    )

    return {
        "input_ids": input_enc["input_ids"].squeeze(),
        "attention_mask": input_enc["attention_mask"].squeeze(),
        "labels": target_enc["input_ids"].squeeze(),
    }

# Apply tokenizer to the dataset
tokenized_data = [tokenize_row(row) for _, row in df.iterrows()]


c:\Users\Salman Awaise\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
from torch.utils.data import Dataset, DataLoader

class ArxivDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        src = item["input_ids"]
        tgt = item["labels"]

        # Creating masks
        src_mask = item["attention_mask"].unsqueeze(0).unsqueeze(0)
        tgt_mask = subsequent_mask(tgt.size(0)).unsqueeze(0)

        return src, tgt, src_mask, tgt_mask

# Create dataset and dataloader
dataset = ArxivDataset(tokenized_data)
train_loader = DataLoader(dataset, batch_size=1, shuffle=True)


In [17]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Model
vocab_size = tokenizer.vocab_size
model = make_model(vocab_size).to("cuda" if torch.cuda.is_available() else "cpu")

# Loss & Optimizer
loss_fn = nn.NLLLoss(ignore_index=tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 10

for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    model.train()
    total_loss = 0

    for src, tgt, src_mask, tgt_mask in tqdm(train_loader):
        src, tgt, src_mask, tgt_mask = src.to(device), tgt.to(device), src_mask.to(device), tgt_mask.to(device)

        # Shift target for decoder input/output
        tgt_input = tgt[:, :-1]
        tgt_output = tgt[:, 1:]
        tgt_len = tgt_input.size(1)
        tgt_mask = subsequent_mask(tgt_len).to(device)      # shape: [1, tgt_len, tgt_len]
        tgt_mask = tgt_mask.unsqueeze(1)

        out = model(src, tgt_input, src_mask, tgt_mask)
        logits = model.generator(out)

        logits = logits.view(-1, logits.size(-1))
        tgt_output = tgt_output.contiguous().view(-1)

        loss = loss_fn(logits, tgt_output)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Average loss: {avg_loss:.4f}")



Epoch 1/10


100%|██████████| 1000/1000 [13:17<00:00,  1.25it/s]


Average loss: 7.3277

Epoch 2/10


100%|██████████| 1000/1000 [13:14<00:00,  1.26it/s]


Average loss: 5.7716

Epoch 3/10


...

Average loss: 2.6282

Epoch 9/10


100%|██████████| 1000/1000 [13:15<00:00,  1.26it/s]


Average loss: 2.2961

Epoch 10/10


100%|██████████| 1000/1000 [13:18<00:00,  1.25it/s]

Average loss: 2.0062


In [18]:
def greedy_decode(model, src, src_mask, max_len, start_symbol, tokenizer):
    # Encoding source input
    memory = model.encode(src, src_mask)

    # Initializing decoder input with start token
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src).to(src.device)

    for i in range(max_len - 1):
        tgt_mask = subsequent_mask(ys.size(1)).to(src.device).unsqueeze(1)

        # Decoding one step
        out = model.decode(memory, src_mask, ys, tgt_mask)

        # Getting log-probabilities from generator
        prob = model.generator(out[:, -1])
        next_word = torch.argmax(prob, dim=-1).unsqueeze(1)

        # Appending the predicted word to the decoder input
        ys = torch.cat([ys, next_word], dim=1)

        # Optional: stop if EOS token is predicted
        if next_word.item() == tokenizer.eos_token_id:
            break

    return ys


In [19]:
def generate_summary(model, input_text, tokenizer):
    model.eval()

    # Tokenize input abstract
    encoding = tokenizer("summarize: " + input_text,
                         return_tensors="pt",
                         max_length=512,
                         truncation=True,
                         padding="max_length")

    src = encoding["input_ids"].to(next(model.parameters()).device)
    src_mask = encoding["attention_mask"].unsqueeze(0).unsqueeze(0).to(src.device)

    # Decode
    start_symbol = tokenizer.convert_tokens_to_ids("summarize")  # or use tokenizer.bos_token_id if available
    decoded_ids = greedy_decode(model, src, src_mask, max_len=64, start_symbol=start_symbol, tokenizer=tokenizer)

    # Convert token IDs to text
    return tokenizer.decode(decoded_ids[0], skip_special_tokens=True)


In [20]:
sample_abstract = df.iloc[0]["abstract"]
generated_title = generate_summary(model, sample_abstract, tokenizer)
print("Generated Title:", generated_title)


Generated Title: $_$
